### Introduction

Beautifulsoup is a python library that is used to collect data from the web (HTML and XML files).
This is a quick and simple tutorial of using Beautifulsoup and Python requests library to show
how we can read and extract info from the web using python.

### 1. Import the libraries

In [42]:
import pandas as pd
import numpy as np

from urllib.request import urlopen
from bs4 import BeautifulSoup

### 2. Opening and parsing the web URL

In [43]:
# urlopen, as the name suggests opens web URLs

url = 'https://www.centralbank.go.ke/inflation-rates'
html_doc = urlopen(url)

In [44]:
# beautifulsoup passes the HTML doc through a parser that makes it python readable

soup = BeautifulSoup(html_doc, 'lxml')

### 3. Exploring the data

With the simple steps above you have a document that you can navigate through and collect whatever data you want from.
Below are some simple things you can extract from the soup structure above.
1. titles - </title > 
2. links - 'a'
3. rows - 'tr-
4. column headers - 'th' 

In [45]:
#Getting the header of the document
title = soup.title
print(title)

<title>Inflation Rates | CBK</title>


In [46]:
#text in the doc remove the (;) to see output
text = soup.get_text()
#print(text)

In [49]:
# Extracting urls found within the page

all_links = soup.find_all('a')

for link in all_links:
#    print(link.get('href'))   

SyntaxError: unexpected EOF while parsing (<ipython-input-49-41c779628cee>, line 6)

In [50]:
# Rows might be something important to explore if site has tabular data

rows = soup.find_all('tr')
#print(rows[:10])

### 4. Data Cleaning

The site that I chose has tabular data. Below I demonstrate a few pandas and python methods to make that data good for use.

In [53]:
# getting column rows and turning into a pandas dataframe

Columns = soup.find_all('th')
def cleaning(Columns):
    
    ''' A function that takes column headers
    converts to a list, parses through beautiful soup
    and then converts to to a dataframe '''
    
    Headers = []
    col_str = str(Columns)
    Clean = BeautifulSoup(col_str, 'lxml').get_text()
    Headers.append(Clean)
    
    df = pd.DataFrame(Headers)
    
    return df

In [54]:
df = cleaning(Columns)
df.head

<bound method NDFrame.head of                                                    0
0  [Year, Month, Annual Average Inflation, 12-Mon...>

Cleaning the dataframe further

In [55]:
df = df[0].str.split(',', expand = True)
df

,0,1,2,3,4,5,6,7
0,[Year,Month,Annual Average Inflation,12-Month Inflation,Year,Month,Annual Average Inflation,12-Month Inflation]


Rows in tabular data

In [56]:
# Putting the rows into a dataframe

rows = soup.find_all('tr')

def row_cleaner(rows):
    ''' Another cleaning function to clean rows from soup and
    pass them into a pandas dataframe'''
    
    All_rows = [] 
    for row in rows:
        row_td = row.find_all('td')
        str_cells = str(row_td)
        clean2 = BeautifulSoup(str_cells, 'lxml').get_text()
        All_rows.append(clean2)
        
    df1 = pd.DataFrame(All_rows)
    return df1 

In [57]:
# The dataframe

df1 =  row_cleaner(rows)

df1.head(4)

,0
0,[]
1,[]
2,"[2020, September, 5.43, 4.20]"
3,"[2020, August, 5.49, 4.36]"


### 5. Webscrapping complete

At this point alll the webscrapping tasks are complete. All you need is a few pandas and Python cleaning techniques to get the data ready for use.
for more about webscrapping, here are the documntations.
1. [Beutifulsoup and parser](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser)
2. [urlib.request library](https://docs.python.org/3.4/library/urllib.request.html)

In [58]:
# Separating the string data to columns
df1 = df1[0].str.split(',', expand = True)
df1.head(5)

,0,1,2,3
0,[],None,None,None
1,[],None,None,None
2,[2020,September,5.43,4.20]
3,[2020,August,5.49,4.36]
4,[2020,July,5.63,4.36]


Joining the two dataframes.

In [59]:
frames = [df, df1]

df2 = pd.concat(frames)

df2.head(5)


,0,1,2,3,4,5,6,7
0,[Year,Month,Annual Average Inflation,12-Month Inflation,Year,Month,Annual Average Inflation,12-Month Inflation]
0,[],None,None,None,NaN,NaN,NaN,NaN
1,[],None,None,None,NaN,NaN,NaN,NaN
2,[2020,September,5.43,4.20],NaN,NaN,NaN,NaN
3,[2020,August,5.49,4.36],NaN,NaN,NaN,NaN


Delete unwanted columns, from 4 to 7

In [60]:
# Deleting unwated columns

df2 = df2.drop(columns = [4, 5, 6, 7])
df2.head(5)

,0,1,2,3
0,[Year,Month,Annual Average Inflation,12-Month Inflation
0,[],None,None,None
1,[],None,None,None
2,[2020,September,5.43,4.20]
3,[2020,August,5.49,4.36]


In [61]:
# Promote first row to columns
df2 = df2.rename(columns = df2.iloc[0])
df2.head()

,[Year,Month,Annual Average Inflation,12-Month Inflation
0,[Year,Month,Annual Average Inflation,12-Month Inflation
0,[],None,None,None
1,[],None,None,None
2,[2020,September,5.43,4.20]
3,[2020,August,5.49,4.36]


In [62]:
# List columns, to ensure names don't have unclear characters like spaces
df2.columns

Index(['[Year', ' Month', ' Annual Average Inflation', ' 12-Month Inflation'], dtype='object')

In [63]:
# Renaming columns

df2.rename(columns = {'[Year': 'Year',
                      ' Month': 'Month',
                      ' Annual Average Inflation': 'Annual_Average_Inflation',
                      ' 12-Month Inflation': 'Twelve_Month_Inflation'}, inplace = True)
df2.head()


,Year,Month,Annual_Average_Inflation,Twelve_Month_Inflation
0,[Year,Month,Annual Average Inflation,12-Month Inflation
0,[],None,None,None
1,[],None,None,None
2,[2020,September,5.43,4.20]
3,[2020,August,5.49,4.36]


In [64]:
# Drop first threee rows
df2 = df2.drop([0,1], axis = 0)
df2.head(3)

,Year,Month,Annual_Average_Inflation,Twelve_Month_Inflation
2,[2020,September,5.43,4.20]
3,[2020,August,5.49,4.36]
4,[2020,July,5.63,4.36]


In [65]:
#Further cleaning

df2['Year'] = df2['Year'].str.strip('[')
df2['Twelve_Month_Inflation'] = df2['Twelve_Month_Inflation'].str.strip(']')
df2.head(3)

,Year,Month,Annual_Average_Inflation,Twelve_Month_Inflation
2,2020,September,5.43,4.20
3,2020,August,5.49,4.36
4,2020,July,5.63,4.36


In [66]:
df2.shape

(190, 4)

In [67]:
df2.dtypes

Year                        object
Month                       object
Annual_Average_Inflation    object
Twelve_Month_Inflation      object
dtype: object

In [70]:
# Change datatypes
df2 = df2.astype({'Annual_Average_Inflation': float, 'Twelve_Month_Inflation': float})
df2.dtypes

Year                         object
Month                        object
Annual_Average_Inflation    float64
Twelve_Month_Inflation      float64
dtype: object

In [71]:
# Removing whitespace
def Clean(x):
    x = x.replace(' ', '')
    return str(x)
df2['Month'] = df2['Month'].apply(Clean)

Converting month into numeric so that we can convert month and year to date.

In [72]:
# We need to import calendar and datetime
import calendar
from datetime import date

df2['Month'] = pd.to_datetime(df2.Month, format='%B').dt.month #too_datetime coverts word dates to numeric dates
df2.head()


,Year,Month,Annual_Average_Inflation,Twelve_Month_Inflation
2,2020,9,5.43,4.20
3,2020,8,5.49,4.36
4,2020,7,5.63,4.36
5,2020,6,5.76,4.59
6,2020,5,5.73,5.33


Create a date column.

In [74]:
# Create datetime and set as index
import calendar
from datetime import date

df2['DATE'] = pd.to_datetime(df2[['Year', 'Month']].assign(DAY=1)) # assigned arbitrary date
df2 = df2.set_index('DATE')
df2.head()

,Year,Month,Annual_Average_Inflation,Twelve_Month_Inflation
DATE,,,,
2020-09-01,2020,9,5.43,4.20
2020-08-01,2020,8,5.49,4.36
2020-07-01,2020,7,5.63,4.36
2020-06-01,2020,6,5.76,4.59
2020-05-01,2020,5,5.73,5.33


Exporting

In [74]:
#df2.to_csv("output_excel_file.xlsx", sheet_name = 'Inflation_rates', index=False)
df2.to_csv("Ke_Inflation.csv", index=False, encoding='utf8')

### 6. Conslusion

This only a simple demonstration of webscrapping and cleaning, for more webscrapping usage you can check the documentations below.
1. [BeautifulSoup Library](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser)
2. [urlib.request Library](https://docs.python.org/3.4/library/urllib.request.html)